In [2]:
from IPython.core.display import HTML

HTML("<style>" + open("style.css").read() + "</style>")

<div class="headline">
Language Technology / Sprachtechnologie
<br><br>
Wintersemester 2019/2020
</div>
<br>
<div class="description">
    Übung zum Thema <i id="topic">"Part-of-speech-tagging"</i>
    <br><br>
    Deadline Abgabe: <i #id="submission">Thursday, 07.11.1019 (23:55 Uhr)</i>
</div>

In [3]:
import nltk
from nltk.probability import FreqDist
from nltk.probability import ConditionalFreqDist
from nltk.corpus import*
from nltk.book import*
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize


*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


# Präsenzübung




### Warm up

<div class="task_description">
    <i class="task">Task 3.1:</i> <br>
</div>


Discuss which of the following concepts may be regarded as POS tags.
1. word classes
2. lexical categories
3. verbs, nouns, adjectives
4. present tense, past tense
5. emotion



<strong style="color: blue">Lösung:</strong>

1. True. POS tags are basically word classes.
2. True, because lexical category is considered to be a synonym for word class.
3. True, these word classes are standard POS.
4. False. Some POS tagsets (e.g. Penn Treebank tagset) have subcategories for verbs that code tense but tense is not itself a part of speech.
5. No, because “emotion” is not considered to be a word class.

<div class="task_description">
    <i class="task">Task 3.2:</i> <br>
</div>

Which of the following statements are true? Find examples or counterexamples.
1. Homographs may be used as synonyms.
2. Homographs remain homographs regardless of their capitalization.
3. Homographs belong to the same word class.
4. Homographs have the same pronunciation.
5. Homographs are spelled equally. 


<strong style="color: blue">Lösung:</strong>

1. False. Homographs have different meaning by definition, e.g. ’bat’ (an animal) vs ’bat’ (hitting instrument)
2. False. As capitalization in English usually changes the meaning of a word, differently capitalized words are not considered to be spelled equally.
3. Not necessarily. For example, the both words “bat” (an animal) and “bat” (hitting instrument) are nouns, while there also is the verb “bat” (strike with, or as if with a baseball bat) as in “bat the ball”.
4. Not necessarily, e.g. for example “conflict” (verb) has an accent on the second syllable, while “conflict” (noun) has accent on the first syllable. If two homographs have the same pronunciation, they are called homonyms.
5. True, by definition. Words with different spelling and same pronunciation are called homophones, for example too, two and to.

<div class="task_description">
    <i class="task">Task 3.3:</i> <br>
</div>

N-gram tagging: Which of the following statements are true?

1. A trigram tagger is theoretically more accurate than a bigram tagger.
2. A trigram tagger looks at the current token and two preceding tags.
3. At sentence boundaries, the bigram tagger takes the two tokens of the previous sentence.

<strong style="color: blue">Lösung:</strong>

1. Yes, in theory, a trigram tagger considers the context of three words to perform the tagging, which should be more accurate than considering only two words like the bigram tagger. However, in practice, the trigram tagger needs a very, very large training corpus to obtain reliable frequency estimates for all possible trigrams. Thus, the accuracy of a trigram tagger trained on a limited corpus might be worse than those of a bigram tagger. However, in practice, “backoff” is used. Thus, the trigram tagger falls back to the bigram tagger in situations where there is not enough training data for a certain trigram.
2. True. However, it is also possible to consider both left and right context.
3. False, because the tokens in the previous sentences do not help to identify a correct tag for the current token. Usually special tokens representing “Begin-of-Sentence” (BOS) are used.

<div class="task_description">
    <i class="task">Task 3.4:</i> <br>
</div>

Confusion matrix: Which of the following statements are true?
1. The diagonal elements are always 1.
2. Cell (i , j) contains the count of the number of times tag i were classified as tag j.
3. The confusion matrix may be used to identify tagging problems.
4. The confusion matrix may be used to compare different taggers.
5. The number of rows is equal to the number of columns.

<strong style="color: blue">Lösung:</strong>

1. False, because the diagonal elements represent the correctly classified instances.
2. Correct by the definition of the confusion matrix.
3. True, because it helps us to determine how often some label i was wrongly tagged with a label j.
4. True, because we may compare the number of errors produced by two taggers.
5. True. cell[i, j] demonstrates how many labels i were assigned with a tag j, hence the number of horizontal and vertical rows is equal.

<div class="task_description">
    <i class="task">Task 3.5:</i> <br>
</div>

Given the sentence “The near future promises happiness for all students.”. Tag it manually (using universal tags) and explain which of the following clues were helpful:
1. morphological clues
2. syntactic clues
3. semantic clues

<strong style="color: blue">Lösung:</strong>

1. The suggested tagging is “The/DET near/ADJ future/NOUN promises/VERB happyness/NOUN for/ADP all/DET students/NOUN ./.”. Morphological clues are useful to determine the part-of-speech of happyness as word with the suffix “-ness” are usually nouns (there are some exceptions, e.g. “to witness”).

In [4]:
pos_tag(word_tokenize("The near future promises happiness for all students."), tagset='universal')

[('The', 'DET'),
 ('near', 'ADJ'),
 ('future', 'NOUN'),
 ('promises', 'VERB'),
 ('happiness', 'NOUN'),
 ('for', 'ADP'),
 ('all', 'DET'),
 ('students', 'NOUN'),
 ('.', '.')]

2. Syntactical clues are useful to disambiguate that future is a noun (it is found in a DET + ADJ + NOUN structure), while promises is a verb (it is found between two noun phrases).
3. Semantic clues are not of much use here.

### Using a tagger

<div class="task_description">
    <i class="task">Task 3.6:</i> <i class="l1">L1</i> <br>
</div>

Many words, like “ski” and “race”, can be used as nouns or verbs with no difference in pronunciation. Can you think of others? <br> Hint: Think of a commonplace object and try to put the word “to” before it to see if it can also be a verb, or think of an action and try to put “the” before it to see if it can also be a noun. Now make up a sentence with both uses of this word, and run the POS-tagger from NLTK on this sentence (using the universal tagset).

<strong style="color: blue">Lösung:</strong>

The phenomenon when a verb becomes noun is called nominalization. Nominalization without any overt changes to the word (called "conversion") is very common in English. However, in other languages (for example Russian) verbs and nouns have different suffixes and this phenomenon is very rare. <br>Examples of English nominalized words are "attack" or "sentence". A recent example is “google” that is now used as a verb, too. <br>Example sentences and their part-of-speech (POS) tags obtained using
nltk.pos_tag(nltk.word_tokenize(sentence), tagset="universal")
 are given in the following table <br>




In [5]:
l=["A bicyle race will be nice.\nLet's race and see who gets there first.","His sentence was 5 to 10 years.\n A judge will sentence him to ten years in prison.","Google is a search engine that searches for men.\n She will google the man she had met at the party"]
for i in range (3):
    tok = nltk.pos_tag(nltk.word_tokenize(l[i]), tagset="universal")
    print(l[i],"\n")
    for i in tok:
        print(i[0],"-->",i[1],"\n")

A bicyle race will be nice.
Let's race and see who gets there first. 

A --> DET 

bicyle --> ADJ 

race --> NOUN 

will --> VERB 

be --> VERB 

nice --> ADJ 

. --> . 

Let --> VERB 

's --> PRT 

race --> NOUN 

and --> CONJ 

see --> VERB 

who --> PRON 

gets --> VERB 

there --> ADV 

first --> ADJ 

. --> . 

His sentence was 5 to 10 years.
 A judge will sentence him to ten years in prison. 

His --> PRON 

sentence --> NOUN 

was --> VERB 

5 --> NUM 

to --> PRT 

10 --> NUM 

years --> NOUN 

. --> . 

A --> DET 

judge --> NOUN 

will --> VERB 

sentence --> VERB 

him --> PRON 

to --> PRT 

ten --> VERB 

years --> NOUN 

in --> ADP 

prison --> NOUN 

. --> . 

Google is a search engine that searches for men.
 She will google the man she had met at the party 

Google --> NOUN 

is --> VERB 

a --> DET 

search --> NOUN 

engine --> NOUN 

that --> DET 

searches --> VERB 

for --> ADP 

men --> NOUN 

. --> . 

She --> PRON 

will --> VERB 

google --> VERB 

the --> DET 


| words | POS   | Sentence | Parsed Sentence
|------|------|------|-----
|  race <br> race |noun <br> verb | A bicycle race will be nice <br> Let’s race and see who gets there first |[(’A’, ’DET’), (’bicycle’, ’ADJ’), (’race’, ’NOUN’), (’will’, ’VERB’), (’be’, ’VERB’), (’nice’, ’ADJ’)] <br> [(’Let’, ’VERB’), (“s", ’PRT’), (’race’, ’NOUN’), (’and’, ’CONJ’), (’see’, ’VERB’), (’who’, ’PRON’), (’gets’, ’VERB’), (’there’, ’ADV’), (’first’, ’ADJ’)]
| sentence <br> sentence | noun <br> verb | His sentence was 5 to 10 years. <br> A judge will sentence him to ten years in prison | [(’His’, ’PRON’), (’sentence’, ’NOUN’), (’was’, ’VERB’), (’5’, ’NUM’), (’to’, ’PRT’), (’10’, ’NUM’), (’years’, ’NOUN’), (’.’, ’.’)] <br> [(’A’, ’DET’), (’judge’, ’NOUN’), (’will’, ’VERB’), (’sentence’, ’VERB’), (’him’, ’PRON’), (’to’, ’PRT’), (’ten’, ’VERB’), (’years’, ’NOUN’), (’in’, ’ADP’), (’prison’, ’NOUN’), ('.', '.')]
| google <br> google | noun <br> verb | Google is a search engine that searches for men. <br> She will google the man she had met at the party | [(’Google’, ’NOUN’), (’is’, ’VERB’), (’a’, ’DET’), (’search’, ’NOUN’), (’engine’, ’NOUN’), (’that’, ’DET’), (’searches’, ’VERB’), (’for’, ’ADP’), (’men’, ’NOUN’), (’.’, ’.’)] <br> [(’She’, ’PRON’), (’will’, ’VERB’), (’google’, ’VERB’), (’the’, ’DET’), (’man’, ’NOUN’), (’she’, ’PRON’), (’had’, ’VERB’), (’met’, ’VERB’), (’at’, ’ADP’), (’the’, ’DET’), (’party’, ’NOUN’)]


The verb “race” is incorrectly tagged as “NOUN” in the second sentence.

<div class="task_description">
    <i class="task">Task 3.7:</i> <br>
</div>

Different words that have the same written form are called homographs. A simplified definition is that homographs are words with the same spelling (case sensitive) and different POS. <br><br>

<div class="task_description">
   <i class="subtask">3.7.1</i> <i class="l1">L1</i> <br>
</div>

Take a look at the function below, what will be printed on the console if it is executed?

In [8]:
tag_fd = nltk.FreqDist(tag for (word, tag) in brown.tagged_words(tagset='universal'))
print(tag_fd.N())

1161192


<strong style="color: blue">Lösung:</strong>

The function will extract all tagged tokens from the corpus and includes each in a frequency distribution with the token and its universal tag. The output contains the total number of sample outcomes that have been recorded by the frequency distribution.

<div class="task_description">
   <i class="subtask">3.7.2</i> <i class="l2">L2</i> <br>
</div>

As the function above is quite useless in its current form, it needs to be improved. Do so by changing it so that the output contains all POS tags (universal) that are used in a corpus.

<strong style="color: blue">Lösung:</strong>

In [6]:
tag_fd = nltk.FreqDist(tag for (word, tag) in brown.tagged_words(tagset='universal'))
print(tag_fd.keys())

dict_keys(['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X'])


For the Brown Corpus: DET,NOUN,ADJ,VERB,ADP,.,ADV,CONJ,PRT,PRON,NUM,X

<div class="task_description">
   <i class="subtask">3.7.3</i> <i class="l3">L3</i> <br>
</div>

Change the function again so that it finds all homographs (using the universal definition) within a corpus. 

<strong style="color: blue">Lösung:</strong>

To see that we really use a simplification for homographs, type the word “bass” into www.leo.org and check the different meanings of the listed nouns. Solution code:

In [7]:
cfd = nltk.ConditionalFreqDist((word, tag) for word, tag in brown.tagged_words(tagset='universal'))

homographs = [key for key in cfd.keys() if len(cfd[key]) > 1]

for homograph in homographs:
    print(homograph)
    print(cfd[homograph].keys())
    print()

Grand
dict_keys(['ADJ', 'X'])

an
dict_keys(['DET', 'X', 'CONJ'])

of
dict_keys(['ADP', 'X'])

primary
dict_keys(['NOUN', 'ADJ'])

no
dict_keys(['DET', 'ADV'])

evidence
dict_keys(['NOUN', 'VERB'])

that
dict_keys(['ADP', 'PRON', 'DET', 'ADV', 'X'])

any
dict_keys(['DET', 'ADV'])

place
dict_keys(['NOUN', 'VERB'])

further
dict_keys(['ADV', 'ADJ', 'VERB'])

in
dict_keys(['ADP', 'PRT', 'NOUN', 'X'])

the
dict_keys(['DET', 'X'])

Executive
dict_keys(['ADJ', 'NOUN'])

,
dict_keys(['.', 'X'])

which
dict_keys(['DET', 'X'])

had
dict_keys(['VERB', 'X'])

charge
dict_keys(['NOUN', 'VERB'])

praise
dict_keys(['NOUN', 'VERB'])

and
dict_keys(['CONJ', 'X'])

for
dict_keys(['ADP', 'ADV', 'NOUN'])

term
dict_keys(['NOUN', 'VERB'])

by
dict_keys(['ADP', 'ADV', 'X'])

Judge
dict_keys(['NOUN', 'VERB'])

to
dict_keys(['PRT', 'ADP', 'NOUN', 'X', 'ADV'])

reports
dict_keys(['NOUN', 'VERB'])

Only
dict_keys(['ADV', 'ADJ'])

a
dict_keys(['DET', 'X', 'NOUN'])

relative
dict_keys(['ADJ', 'NOUN'])

such
dic

<div class="task_description">
    <i class="task">Task 3.8:</i> <br>
</div>

Write programs to process the Brown Corpus and find answers to the following questions:

<div class="task_description">
   <i class="subtask">3.8.1</i> <i class="l1">L1</i> <br>
</div>

What is the output of the function below? Proof your answer by executing it on your computer.

In [8]:
n=FreqDist([word for (word,tag) in brown.tagged_words(tagset='universal') if tag == 'NOUN'])
print ([i for i in n])

['time', 'man', 'Af', 'years', 'way', 'Mr.', 'people', 'men', 'world', 'life', 'year', 'day', 'work', 'Mrs.', 'home', 'state', 'one', 'place', 'number', 'course', 'part', 'fact', 'States', 'water', 'something', 'school', 'hand', 'head', 'night', 'system', 'house', 'eyes', 'days', 'group', 'program', 'room', 'side', 'business', 'things', 'end', 'John', 'nothing', 'case', 'use', 'point', 'order', 'children', 'thing', 'power', 'interest', 'area', 'face', 'problem', 'members', 'country', 'door', 'family', 'development', 'war', 'York', 'sense', 'form', 'God', 'kind', 'others', 'action', 'mind', 'example', 'hands', 'feet', 'matter', 'line', 'law', 'government', 'name', 'car', 'words', 'body', 'history', 'anything', 'death', 'State', 'period', 'word', 'times', 'city', 'money', 'information', 'week', 'President', 'experience', 'light', 'today', 'moment', 'service', 'field', 'problems', 'job', 'reason', 'areas', 'position', 'boy', 'question', 'Miss', 'wife', 'church', 'policy', 'voice', 'air', 

<strong style="color: blue">Lösung:</strong>

The function prints each token from the corpus that is tagged as a noun in the console.

___

Different corpora don't necessarily employ the same tagset. In the following we will inspect the brown corpus and its original tagset. 

<div class="task_description">
   <i class="subtask">3.8.2</i> <i class="l2">L2</i> <br>
</div>

What are the 10 most frequent tags in the Brown Corpus using the original Brown tagset (instead of the universal tagset).

In [9]:
tag_fd_br = FreqDist(tag for (word, tag) in brown.tagged_words())
print ((tag_fd_br.most_common(10)))

[('NN', 152470), ('IN', 120557), ('AT', 97959), ('JJ', 64028), ('.', 60638), (',', 58156), ('NNS', 55110), ('CC', 37718), ('RB', 36464), ('NP', 34476)]


<div class="task_description">
   <i class="subtask">3.8.3</i> <i class="l2">L2</i> <br>
</div>

What do these tags stand for? Print 10 example words for each tag.

In [10]:
for y in (tag_fd_br.most_common(10)):
    fd_word_br = FreqDist(word for (word, tag) in brown.tagged_words()if tag == y[0])
    print(y[0],"example words\n",[i[0] for i in fd_word_br.most_common(10)])

NN example words
 ['time', 'man', 'Af', 'way', 'world', 'life', 'year', 'day', 'work', 'state']
IN example words
 ['of', 'in', 'to', 'for', 'with', 'on', 'by', 'at', 'from', 'into']
AT example words
 ['the', 'a', 'The', 'an', 'no', 'A', 'every', 'No', 'An', 'Every']
JJ example words
 ['new', 'such', 'own', 'good', 'great', 'old', 'long', 'small', 'American', 'high']
. example words
 ['.', ';', '?', '!', ':']
, example words
 [',', ':', '[', ']']
NNS example words
 ['years', 'people', 'men', 'eyes', 'days', 'things', 'children', 'members', 'others', 'hands']
CC example words
 ['and', 'or', 'but', 'But', 'And', 'either', 'nor', 'Or', 'neither', 'Yet']
RB example words
 ['only', 'now', 'then', 'also', 'even', 'back', 'still', 'just', 'never', 'well']
NP example words
 ['Mr.', 'Mrs.', 'John', 'God', 'Miss', 'Washington', 'William', 'Kennedy', 'Congress', 'America']


<div class="task_description">
   <i class="subtask">3.8.4</i> <i class="l2">L2</i> <br>
</div>

Change the function from task 3.8.1 so that the output contains the 5 most frequent singular and the 5 most frequent plural nouns from the corpus.

<strong style="color: blue">Lösung:</strong>

In [11]:
n=FreqDist([word for (word,tag) in brown.tagged_words() if tag == 'NN'])
ns=FreqDist([word for (word,tag) in brown.tagged_words() if tag == 'NNS'])

print ("Singular:",n.most_common(5))
print ("Singular:",ns.most_common(5))

Singular: [('time', 1547), ('man', 1144), ('Af', 990), ('way', 882), ('world', 682)]
Singular: [('years', 939), ('people', 801), ('men', 733), ('eyes', 391), ('days', 376)]



For the Brown Corpus, the output is: Singular: [time, man, Af, way, world], Plural: [years, people, men, States, eyes]

<div class="task_description">
   <i class="subtask">3.8.5</i> <i class="l3">L3</i> <br>
</div>

Which nouns are more common in their plural form, rather than their singular form? Change the function again to find out! (Only consider regular plurals, formed with the -s suffix.) 

<strong style="color: blue">Lösung:</strong>

Refer to the code in the following listing:

In [12]:
corpus = brown.tagged_words()
fdistSg=nltk.FreqDist([word for (word,tag) in corpus if tag == 'NN'])
fdistPl=nltk.FreqDist([word for (word,tag) in corpus if tag == 'NNS' and word [-1] == "s"])

for word in fdistPl.keys():
    if fdistPl[word] > fdistSg[word[:-1]]:
        print(word)

irregularities
presentments
thanks
voters
legislators
topics
policies
steps
funds
items
counties
taxpayers
appraisers
guardians
fees
juries
citizens
servants
concessionaires
deputies
officials
employes
Attorneys
candidates
relations
years
bonds
sales
ones
authorities
allowances
members
bankers
rules
witnesses
dollars
means
companies
obligations
depositors
customers
millions
Senators
students
savings
scholastics
parimutuels
ponies
cities
senators
retailers
taxes
brokers
vacancies
representatives
two-thirds
majorities
Opponents
Statements
defenders
methods
hours
co-signers
regents
relatives
parents
defendants
observers
others
advisers
proceedings
subpenas
processes
workers
Officials
requirements
facilities
elements
Legislators
ayes
noes
difficulties
tragedies
allies
circumstances
critics
manifestations
arms
speeches
events
months
submarines
societies
countries
modifications
errors
conditions
victories
channels
Policies
negotiations
losses
neighbors
communities
residents
sirens
intervals


<div class="task_description">
    <i class="task">Task 3.9:</i> <i class="l3">L3</i> <br>
</div>

In the Brown corpus, what percentage of words are tagged with the first five most common original tags?

<strong style="color: blue">Lösung:</strong>



In [13]:
tag_fd = nltk.FreqDist(tag for (word, tag) in brown.tagged_words())
sum=0
for i in tag_fd.most_common(5):
   sum+= i[1]
print(sum/tag_fd.N())

0.4268475842065739


The first five tags cover about 43% of all tokens.

<div class="task_description">
    <i class="task">Task 3.10:</i> <br>
</div>

Generate some statistics for tagged data to answer the following questions:

<div class="task_description">
   <i class="subtask">3.10.1</i> <i class="l1">L1</i> <br>
</div>
Take a look at the code below. What does it compute? 

In [14]:
cfd = nltk.ConditionalFreqDist([(word.lower(),tag) for (word, tag) in brown.tagged_words()])
nr_tags_cfd = nltk.ConditionalFreqDist([(len(cfd[word]), word) for word in cfd.conditions()])

nr_of_tags = 1
print("words with one tag:", len(nr_tags_cfd[nr_of_tags]))
print("ratio:", float(len(nr_tags_cfd[nr_of_tags]))/len(cfd.conditions()))

words with one tag: 40235
ratio: 0.8076884472548429


<strong style="color: blue">Lösung:</strong>

The Code computes the proportion of how many word types are always assigned the same POS tag. For the Brown corpus, there are 40235 types that appear with only one (non-universal) POS-tag. This are 81% of all word types in the corpus.

<div class="task_description">
   <i class="subtask">3.10.2</i> <i class="l2">L2</i> <br>
</div>

How many words are ambiguous, in the sense that they appear with at least two tags (no programming required)? Change the function above so that it computes the percentage of word tokens that involve these ambiguous words.

<strong style="color: blue">Lösung:</strong>

In [ ]:
cfd = nltk.ConditionalFreqDist([(word.lower(),tag) for (word, tag) in brown.tagged_words()])
nr_tags_cfd = nltk.FreqDist([len(cfd[word]) for word in brown.words()])

print("ambigious words:", len(brown.words()) - nr_tags_cfd[1])
print("ratio:", float(1-nr_tags_cfd[1]/len(brown.words())))

19% of the words in the corpus are ambiguous (can simply induced from 3.10.1 without any programming). The overall ratio of ambiguous tokens in the Brown Corpus is 87%. This means that a small amount of 19% ambiguous types correspond to a large number of ambiguous tokens. It is a known linguistic phenomenon that frequent words are more ambiguous than seldom occurring words (see Zipf distribution in lecture).

<div class="task_description">
   <i class="subtask">3.10.3</i> <i class="l3">L3</i> <br>
</div>

Which words in the Brown corpus have the highest number of distinct tags?

<strong style="color: blue">Lösung:</strong>

In [15]:
cfd= nltk.ConditionalFreqDist((word.lower(),tag) for (word,tag) in brown.tagged_words())
nr_tags_cfd = nltk.ConditionalFreqDist([(len(cfd[word]), word) for word in cfd.conditions()])

highest_nr_tags=max(nr_tags_cfd.conditions())
print("Highest # of tags:", highest_nr_tags)
print("Words with ",highest_nr_tags,"tags: ", ', '.join(nr_tags_cfd[highest_nr_tags].keys()))
for word in nr_tags_cfd[highest_nr_tags].keys():
    print (word, "-->", ' '.join(cfd[word]))

Highest # of tags: 15
Words with  15 tags:  that
that --> CS DT WPS WPO QL DT-NC DT-TL WPS-TL WPS-NC CS-NC WPS-HL CS-HL DT-HL NIL WPO-NC


The highest number of tags is 15. There is one word with 15 different tags: that

# Automatic Tagging

<div class="task_description">
    <i class="task">Task 3.11:</i> <br>
</div>

<div class="task_description">
   <i class="subtask">3.11.1</i> <i class="l1">L1</i> <br>
</div>

Look at the code below, explain.



In [ ]:
wordTagCfd = nltk.ConditionalFreqDist([(word.lower(),tag) for (word, tag) in brown.tagged_words(tagset='universal')])

getASentence(brown.tagged_words)

def getASentence():
    #TODO
def getMostLikelyTag():
    #TODO

<strong style="color: blue">Lösung:</strong>

A conditional frequency distribution ”wordTagCfd” is created, that includes each word of the brown corpus together with the universal tag of it.

<div class="task_description">
   <i class="subtask">3.11.2</i> <i class="l2">L2</i> <br>
</div>

We now want to give the code a little bit more functionality. As you can see, the method getASentence is called, but at the moment, it will not do anything. Enhance it so that when called, it will print out each token of a sentence in the Brown corpus (Hint: getASentence(40) will return sentence number 41, not 40).

<strong style="color: blue">Lösung:</strong>



In [16]:
wordTagCfd = nltk.ConditionalFreqDist([(word.lower(),tag) for (word, tag) in brown.tagged_words(tagset='universal')])


def getASentence(nr):
    x = brown.tagged_sents(tagset='universal')[nr]
    for i in x:
        print (i[0])
def getMostLikelyTag():
    #TODO
    
getASentence(40)



IndentationError: expected an indented block (<ipython-input-16-945e35a78540>, line 11)

For sentence 41:
Hartsfield has
been mayor
of Atlanta
,
with exception of
one
brief interlude ,
since 1937
.

<div class="task_description">
   <i class="subtask">3.11.3</i> <i class="l3">L3</i> <br>
</div>

We now want the tag for each token of the sentence. Enhance the method getMostLikelyTag so that the most likely tag for a token is returned (use the wordTagCfd as a lookup dictionary and test your method on a sentence from the Brown corpus).
Then change the method getASentence again so that not only each token, but also the tag of the token is printed out in the console.
Test your method on a sentence from the Moby Dick corpus. How well does it perform? 

In [17]:
wordTagCfd = ConditionalFreqDist([(word,tag)
    for (word, tag) in brown.tagged_words(tagset='universal')])

def getASentence(x):
    for i in x:
        print(i,'-',get_most_likely_tag(i,wordTagCfd))
def get_most_likely_tag(word,lookup_cfd):
    tag='UNK'
    if word in lookup_cfd.conditions():
        tag=lookup_cfd[word].max()
    return tag


print("Brown: ",getASentence(brown.sents()[40]))
print("Moby: ",getASentence(text1[42:73]))

Hartsfield - NOUN
has - VERB
been - VERB
mayor - NOUN
of - ADP
Atlanta - NOUN
, - .
with - ADP
exception - NOUN
of - ADP
one - NUM
brief - ADJ
interlude - NOUN
, - .
since - ADP
1937 - NUM
. - .
Brown:  None
He - PRON
was - VERB
ever - ADV
dusting - VERB
his - DET
old - ADJ
lexicons - UNK
and - CONJ
grammars - UNK
, - .
with - ADP
a - DET
queer - ADJ
handkerchief - NOUN
, - .
mockingly - ADV
embellished - VERB
with - ADP
all - PRT
the - DET
gay - ADJ
flags - NOUN
of - ADP
all - PRT
the - DET
known - VERB
nations - NOUN
of - ADP
the - DET
world - NOUN
. - .
Moby:  None


<strong style="color: blue">Lösung:</strong>

Several tokens from the Moby Dick corpus have no tag assigned to them (marked with UNK), as they did not appear in the data used for training the tagger (Brown corpus).

<div class="task_description">
    <i class="task">Task 3.12:</i> <br>
</div>

Explore the n-gram taggers for n ∈ 1, 2, 3 in the following listing:

In [20]:
t0=nltk.DefaultTagger('NOUN')
train_sents=brown.tagged_sents(tagset='universal')
t1 = nltk.UnigramTagger(train_sents)
t2 = nltk.BigramTagger(train_sents)
t3 = nltk.TrigramTagger(train_sents)
print(t2)

<BigramTagger: size=131263>


<div class="task_description">
   <i class="subtask">3.12.1</i><br>
</div>

Manually tag the sentence “The only conservative councillor representing Cambridge resigned from the city council.” using the universal tagset. Represent it as a tagged sentence.

<strong style="color: blue">Lösung:</strong>

|Word|Correct
|-|-
|The|DET
|only | ADJ
|conservative |ADJ
|councillor |NOUN
|representing |VERB
|Cambridge | NOUN
|resigned| VERB
|from |ADP
|the| DET
|city |NOUN
|council|NOUN
|.|.


<div class="task_description">
   <i class="subtask">3.12.2</i> <i class="l2">L2</i> <br>
</div>

We now want to evaluate the performance of different n gram taggers. Enhance the the code using the function tagger.evaluate(sentences) so that all four taggers are evaluated on the sentence from task 2.12.1 The default tagger is already implemented.

<strong style="color: blue">Lösung:</strong>

In [23]:
train_sents = brown.tagged_sents(tagset='universal')
t1 = nltk.UnigramTagger(train_sents)
t2 = nltk.BigramTagger(train_sents)
t3 = nltk.TrigramTagger(train_sents)

sentence = ['The' , 'only', 'conservative' , 'councillor' , 'representing' , 'Cambridge' , 'resigned' , 'from' , 'the' , 'city' , 'council' , '.']
tagged_sentence = [('The', 'DET') , ('only', 'ADJ'), ('conservative','ADJ') , ('councillor','NOUN') , ('representing','VERB') , ('Cambridge','NOUN') ,( 'resigned','VERB') ,( 'from','ADP'), ('the','DET') , ('city','NOUN') , ('council','NOUN') ,( '.','.')]

                                 
print (t0.tag(sentence))
print (t0.evaluate([tagged_sentence]))
print (t1.tag(sentence))
print (t1.evaluate([tagged_sentence]))
print (t2.tag(sentence))
print (t2.evaluate([tagged_sentence]))
print (t3.tag(sentence))
print (t3.evaluate([tagged_sentence])                                 )

                                 

[('The', 'NOUN'), ('only', 'NOUN'), ('conservative', 'NOUN'), ('councillor', 'NOUN'), ('representing', 'NOUN'), ('Cambridge', 'NOUN'), ('resigned', 'NOUN'), ('from', 'NOUN'), ('the', 'NOUN'), ('city', 'NOUN'), ('council', 'NOUN'), ('.', 'NOUN')]
0.3333333333333333
[('The', 'DET'), ('only', 'ADV'), ('conservative', 'ADJ'), ('councillor', None), ('representing', 'VERB'), ('Cambridge', 'NOUN'), ('resigned', 'VERB'), ('from', 'ADP'), ('the', 'DET'), ('city', 'NOUN'), ('council', 'NOUN'), ('.', '.')]
0.8333333333333334
[('The', 'DET'), ('only', 'ADJ'), ('conservative', 'ADJ'), ('councillor', None), ('representing', None), ('Cambridge', None), ('resigned', None), ('from', None), ('the', None), ('city', None), ('council', None), ('.', None)]
0.25
[('The', 'DET'), ('only', 'ADJ'), ('conservative', 'ADJ'), ('councillor', None), ('representing', None), ('Cambridge', None), ('resigned', None), ('from', None), ('the', None), ('city', None), ('council', None), ('.', None)]
0.25


Most of the bigrams and trigrams in the sentence are not found in the corpus. Thus, the performance of the n-gram taggers decreases with increasing n. 

<div class="task_description">
   <i class="subtask">3.12.3</i> <i class="l3">L3</i> <br>
</div>

The idea of backoff tagging is to rely on other taggers when the current tagger cannot assign a label. For example, a bigram tagger might never encounter some bigram, however a unigram tagger may separately tag the single words in the bigrams. In this task, we will explore the contribution of n-gram taggers for n ∈ 1, 2, 3 in the backoff setup, provided by the following listing:

In [29]:
t0 = nltk.DefaultTagger('NOUN')
train_sents = brown.tagged_sents(tagset='universal')
t1 = nltk.UnigramTagger(train_sents, backoff = t0)
t2 = nltk.BigramTagger(train_sents, backoff = t1)
t3 = nltk.TrigramTagger(train_sents, backoff = t2)

[('The', 'NOUN'), ('only', 'NOUN'), ('conservative', 'NOUN'), ('councillor', 'NOUN'), ('representing', 'NOUN'), ('Cambridge', 'NOUN'), ('resigned', 'NOUN'), ('from', 'NOUN'), ('the', 'NOUN'), ('city', 'NOUN'), ('council', 'NOUN'), ('.', 'NOUN')]
0.3333333333333333
[('The', 'DET'), ('only', 'ADV'), ('conservative', 'ADJ'), ('councillor', 'NOUN'), ('representing', 'VERB'), ('Cambridge', 'NOUN'), ('resigned', 'VERB'), ('from', 'ADP'), ('the', 'DET'), ('city', 'NOUN'), ('council', 'NOUN'), ('.', '.')]
0.9166666666666666
[('The', 'DET'), ('only', 'ADJ'), ('conservative', 'ADJ'), ('councillor', 'NOUN'), ('representing', 'VERB'), ('Cambridge', 'NOUN'), ('resigned', 'VERB'), ('from', 'ADP'), ('the', 'DET'), ('city', 'NOUN'), ('council', 'NOUN'), ('.', '.')]
1.0


Evaluate the sentence “The only Conservative councillor representing Cambridge resigned from the city council.“ using the taggers t0, t1, t2, and t3 with backoff tagging and evaluate the results like in task 3.12.2). Compare the results with and without backoff.

<strong style="color: blue">Lösung:</strong>

The performance of the default tagger is unchanged, as it is as dumb as before :) The unigram tagger improves a little bit, as in case of unknown words it can now backoff to the default tagger. As the probability that unseen words are nouns is quite high, this gives some improvements. The bigram tagger improves dramatically, as it backs off to the unigram tagger when it cannot recognize a bigram. The behavior of the trigram tagger is identical to the bigram tagger.

# Homework

<div class="task_description">
    <i class="task">Homework 3.1:</i>
        ::: 5 Homework points :::</div>
                                
Write code to search the Brown Corpus for particular words and phrases according to tags (universal tagset), to answer the following questions: <br><br>

<div class="task_description">
   <i class="subtask">3.1.1</i>
</div>
Produce an alphabetically sorted list of the distinct words tagged as ADP.

<div class="task_description">
   <i class="subtask">3.1.2</i>
</div>

Identify words that can be plural nouns or third person singular verbs (e.g. deals, flies).

<div class="task_description">
   <i class="subtask">3.1.3</i>
</div>

For the word(s) with the greatest number of distinct tags, print the sentences from the corpus containing the word.

<div class="task_description">
   <i class="subtask">3.1.4</i>
</div>

What is the ratio of masculine to feminine pronouns?

<div class="task_description">
    <i class="task">Homework 3.2:</i>
        ::: 5 Homework points :::</div>
        

Print a table with the integers 1..10 in one column, and the number of types in the corpus having 1..10 distinct tags in the other column.